# Applying Prompt Engineering Strategies

- Strategy 1: Write clear instructions
- Strategy 2: Provide reference text
- Strategy 3: Break tasks into subtasks
- Strategy 4: Give the model time to think
- Strategy 5: Use external tools
- Strategy 6: Test changes systematically

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI()

# Strategy 1: Write clear instructions
system_message = ""
clear_instructions_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": clear_instructions_prompt}
])
print("Strategy 1 - Clear Instructions:\n", response.choices[0].message.content)

In [ ]:
# Strategy 2: Provide reference text
system_message = ""
reference_text_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": reference_text_prompt}
])
print("\nStrategy 2 - Provide Reference Text:\n", response.choices[0].message.content)

In [ ]:
# Strategy 3: Break tasks into subtasks
system_message = ""
subtasks_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": subtasks_prompt}
])
print("\nStrategy 3 - Break Tasks into Subtasks:\n", response.choices[0].message.content)

In [ ]:
# Strategy 4: Give the model time to think
system_message = ""
time_to_think_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": time_to_think_prompt}
])
print("\nStrategy 4 - Give the Model Time to Think:\n", response.choices[0].message.content)

In [ ]:
# Strategy 5: Use external tools (e.g., Python code execution)
system_message = ""
external_tools_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": external_tools_prompt}
])
print("\nStrategy 5 - Use External Tools:\n", response.choices[0].message.content)

In [ ]:
# Strategy 6: Test changes systematically
system_message = ""
test_changes_systematically_prompt = ""
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": test_changes_systematically_prompt}
])
print("\nStrategy 6 - Test Changes Systematically:\n", response.choices[0].essage.content)